In [13]:
import json as js
import pandas as pd
import numpy as np
import geopandas as gpd

In [23]:
# function to return the name of a country when a nationality is provided 
# it is done to add a column with the country name to the imagegallery and merge it the the shapes of countries
def CountryAssign(natvalue):
    '''give a country name when a nationality is provided'''
    try:
        return countriesCap.loc[countriesCap.nationality == natvalue,'countryName'].tolist()[0]
    except:
        return np.nan
    

In [ ]:
# reqding the file with the necessary data for the color vizualization
imagegallery = pd.read_json('imagegallery.json')

In [ ]:
imageExploded = imagegallery.explode('NationalityCleaner').reindex(axis = 'index')

In [ ]:
imageExploded.NationalityCleaner

In [ ]:
nationalities = imageExploded.NationalityCleaner.unique()

In [ ]:
nationalities.shape

In [ ]:
# This file contains the one to one relation between country and nationality
countriesCap = pd.read_json('CleanCountries.json')

In [ ]:
# Creating and fillig the column with country values in the image gallery 
imageExploded['Country'] = imageExploded['NationalityCleaner'].apply(CountryAssign)

In [ ]:
imageExploded = imageExploded.explode('Colors')

In [ ]:
imageExploded

In [ ]:
##################################################################################################################################################################

In [ ]:
# getting the geometries of the countries
countriesGeojson = gpd.read_file('countries.geojson')

In [ ]:
countriesGeojson

In [ ]:
# visual inspection of the country names
for i,x in enumerate(countries):
    print(countries[i]['properties']['ADMIN'])

In [ ]:
for x in imageExploded['Country'].unique():
    if x in countriesGeojson.ADMIN.tolist():
        continue
    else:
        print(x)

In [ ]:
# these values of countries from the imagegallery did not have a match in the countries and are replaced:  
United States : United States of America
Serbia : Republic of Serbia
Tanzania : United Republic of Tanzania
Bahamas : The Bahamas

In [ ]:
problemNats = ['United States','Serbia','Tanzania','Bahamas']
replacementNats = ['United States of America','Republic of Serbia','United Republic of Tanzania','The Bahamas']

In [ ]:
# changing the problematic nationalities:
imageExploded['Country'] = imageExploded.Country.replace(problemNats,replacementNats)

In [ ]:
imageExploded[imageExploded.Country == 'Republic of Serbia']

In [ ]:
# summary stats 
imageExploded.groupby(['Country']).count()

In [ ]:
# more convenient for the summary statistics
colorMap = imageExploded[['Colors','Country']]
# further add the decade for space time viz

In [ ]:
colorMap

In [ ]:
# pivoting to get the number of artworks by color by country;
colorMap = pd.pivot_table(colorMap,index = ['Country'], columns='Colors',aggfunc=len, fill_value = 0)
# Potentially add the decade for time visualizations
# ,'Decade'

In [ ]:
colorMap

In [ ]:
nats= pd.Series(colorMap.index)

In [ ]:
colorMap.index = range(101)

In [ ]:
colorMap['Country'] = nats

In [ ]:
colorMap

In [ ]:
# THE GEOMETRY ASSIGNEMENT WORKED BU THERE WERE ISSUES WITH LOADING IT INTO MYSQL, SO TRYING ANOTHER METHOD

In [ ]:
colormap = countriesGeojson.merge(colorMap,left_on='ADMIN',right_on = 'Country')

In [ ]:
colormap

In [ ]:
# store data as shapefiles locally 
colormap.to_file('colormap.shp')
colormap.to_file('colormap.geojson',driver='GeoJSON')

In [ ]:
from sqlalchemy import *

In [2]:
import shapely

# The next step is very important before loading the data in mysql. In its current state, the geopandas data frame can not be uploaded to my SQL. 
The trick to overcome this issue is to convert the geometry variable into WKT format for uploading

In [ ]:
prec = 1

In [ ]:
colormap['geometry']

In [ ]:
geom = colormap['geometry']
#colormap

In [ ]:
# Simplifying the geometry to avoid having a heavy data set that would be difficult to visualize on the website. 
geom = geom.apply(lambda x: x.simplify(0.3))

In [ ]:
# rounding the coordinates. 
geom = geom.apply(lambda x: shapely.wkt.dumps(x, rounding_precision=4))

In [ ]:
colormap['geometry'] = geom

In [ ]:
colormap

## WRITING THE DATA TO JSON
Saving the data locally

In [ ]:
colormap = pd.read_json('Colormap.json')

In [ ]:
colormap

In [ ]:
colormap.to_json('colormap.json')

In [ ]:
imageExploded.reindex(axis = 0)

In [ ]:
imageExploded.to_json('imagegallery.json')

## Database Import

We've finally reached a point where we can feel happy with the quality and format our data is in. It's now time to export that data to a database for future use.

We'll use the Pandas SQL functionality to achieve this. Other more comprehensive SQL packages are available, but the Pandas tools make working between dataframes and SQL quite simple.

### Installing Database Package

When accessing a database through an external script or application, you'll always need to ensure two things are done:

1. Make sure the *package* is installed that allows your application (e.g. Python) to talk to your database (e.g. MySQL).

2. Write a *connection string* within your application that tells it where the database is and how to connect to it. The syntax for this varies but the connection string always contains details of the database type, it's location, and the access credentials (e.g. username and password) needed for connection.

**If you're working on your own machine, follow these steps to install the PyMySQL drivers.** This process is the same for any Python package being installed through Anaconda, but you'll find the majority of useful data science libraries come pre-installed. If you're using an ISD machine then unfortunately the installation will not persist outside of your current login, so you'll have to do this again every time you want to use the package.

1. Open Anaconda Navigator and go to the Environments tab.

2. Switch the drop down menu from Installed to All.

3. Search for 'pymysql'.

4. Click on the box on the left-hand side of the 'pymysql' entry, and hit 'Mark for Installation'.

5. Click the Apply button in the bottom right corner, then wait for the installation to complete.

### Making the Database Connection

The Pandas SQL connection builds on the SQLAlchemy library. To create the connection, we first load the SQLAlchemy library, and then create the connection. **The scripts for this are provided below, replace the access credentials where necessary, then run them.** You'll notice that the database location details (host and port numbers) have been provided for you.

The connection settings provided here are for the MySQL database, but different settings are required for other database environments. More information on how to create these connections are provided here: http://docs.sqlalchemy.org/en/rel_0_9/core/engines.html

In [26]:
# import the SQLAlchemy libraries
import sqlalchemy as sqal

In [27]:
# create the connection string to the MySQL database
# replace USERNAME and PASSWORD with your own credentials 
# LUO 
engineLUO = sqal.create_engine('mysql+pymysql://CREDENTIALS@dev.spatialdatacapture.org:3306')
# IVANN 
engine = sqal.create_engine('mysql+pymysql://CREDENTIALS@dev.spatialdatacapture.org:3306')

In [29]:
# make the connection to the database
conn = engine.raw_connection()

OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on 'dev.spatialdatacapture.org' (timed out)")
(Background on this error at: http://sqlalche.me/e/e3q8)

### Accessing the Database from Python

Once we've created the connection to the database, we can write data directly to the database. This uses the `.to_sql()` function on the cleaned dataframe we wish to upload. Within this function we provide parameters the specify the database table name and connection details. The function helpfully creates the new 

**Check out the query below, and then run it.** 

In [6]:
# file for the map vizualisation
colormap.to_sql('colormap',con=engine,if_exists='replace')

NameError: name 'colormap' is not defined

In [ ]:
#countries.to_sql('CountryColormap', con=engine)

In [ ]:
# file for the image gallery
imageExploded.to_sql('artworks', con=engine,if_exists='replace')

Finally, we will want to check that the data has actually been correctly uploaded to the database. You can do this through MySQL Workbench, but it's good to know how to run SQL queries in from Python too. 

Here we will run a Pandas function that will call an SQL query and then place the results within a new dataframe. The function is called `.read_sql()` and just requires the SQL query, and the connection details. 

**Test this out below, filling the SQL query with a simply query on your new dataset. Once you've downloaded the data, check out the contents.**

In [7]:
# remember to enter your SQL query in below
reloadedColorMap = pd.read_sql('SELECT * FROM colormap', conn)

In [30]:
reloadedColorMap

,index,ADMIN,ISO_A3,geometry,aliceblue,antiquewhite,aquamarine,beige,bisque,black,...,silver,snow,tan,teal,tomato,turquoise,violet,wheat,white,yellow
0,0,Afghanistan,AFG,"POLYGON ((71.0498 38.4087, 71.5977 37.8984, 71...",0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,Albania,ALB,"POLYGON ((19.7478 42.5789, 20.5008 42.2112, 20...",0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,2,Argentina,ARG,"MULTIPOLYGON (((-68.6541 -54.8862, -68.6423 -5...",1,37,0,22,5,380,...,72,6,28,3,0,0,0,38,0,0
3,3,Australia,AUS,"MULTIPOLYGON (((158.8657 -54.7499, 158.8525 -5...",0,7,0,9,2,25,...,5,0,2,1,0,0,0,5,0,0
4,4,Austria,AUT,"POLYGON ((15.1618 48.9372, 16.9450 48.6042, 17...",5,64,0,10,8,208,...,98,0,44,2,0,3,0,114,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,96,Uruguay,URY,"POLYGON ((-56.9064 -30.1085, -53.1198 -32.7074...",0,2,0,0,0,48,...,8,1,5,0,0,0,0,2,0,0
97,97,United States of America,USA,"MULTIPOLYGON (((-155.6065 20.1380, -154.8141 1...",105,2126,14,1597,286,11892,...,4440,147,1323,100,14,35,1,1922,16,0
98,98,Venezuela,VEN,"MULTIPOLYGON (((-61.3903 8.5806, -61.3146 8.57...",0,23,0,16,6,141,...,39,11,11,3,0,0,0,19,0,0
99,99,South Africa,ZAF,"MULTIPOLYGON (((37.8638 -46.9409, 37.6188 -46....",0,15,0,26,2,170,...,33,0,4,1,0,0,0,12,0,0


In [19]:
artworks = pd.read_sql('SELECT * FROM artworks', conn)

In [9]:
reloadedartworks

,index,Decade,NationalityCleaner,Colors,ThumbnailURL,URL,Country
0,0,1890,Austrian,rosybrown,http://www.moma.org/media/W1siZiIsIjU5NDA1Il0s...,http://www.moma.org/collection/works/2,Austria
1,1,1980,French,rosybrown,http://www.moma.org/media/W1siZiIsIjk3Il0sWyJw...,http://www.moma.org/collection/works/3,France
2,1,1980,French,dimgray,http://www.moma.org/media/W1siZiIsIjk3Il0sWyJw...,http://www.moma.org/collection/works/3,France
3,2,1900,Austrian,wheat,http://www.moma.org/media/W1siZiIsIjk4Il0sWyJw...,http://www.moma.org/collection/works/4,Austria
4,2,1900,Austrian,rosybrown,http://www.moma.org/media/W1siZiIsIjk4Il0sWyJw...,http://www.moma.org/collection/works/4,Austria
...,...,...,...,...,...,...,...
177630,68692,1920,Russian,black,http://www.moma.org/media/W1siZiIsIjIyNzQ2MiJd...,http://www.moma.org/collection/works/409778,Russia
177631,68693,1920,Russian,gainsboro,http://www.moma.org/media/W1siZiIsIjIyNzQ2MyJd...,http://www.moma.org/collection/works/409779,Russia
177632,68694,1920,Russian,gainsboro,http://www.moma.org/media/W1siZiIsIjIyNzQ2NCJd...,http://www.moma.org/collection/works/409780,Russia
177633,68695,1920,Russian,gainsboro,http://www.moma.org/media/W1siZiIsIjIyNzQ2NSJd...,http://www.moma.org/collection/works/409781,Russia


In [21]:
connLUO = engineLUO.raw_connection()

In [ ]:
reloadedColorMap.to_sql('colormap',con=engineLUO)

In [22]:
reloadedartworks.to_sql('artworks',con=engineLUO,if_exists='replace')

**Open up MySQL Workbench too and go and find your new dataset.**

More information on how to access databases, insert data and query tables using SQL can be found the in documentation here: http://pandas.pydata.org/pandas-docs/stable/io.html#sql-queries

### Exercises

Well done for completing this tutorial. You should now have a good grounding in how to use Python and Pandas to load and clean data, and upload it to a database. We'll continue to use similar methods in the coming weeks. 

If you want to explore these methods further in the meantime, here are a few additional activities you might want to try:

* Download a new dataset from London Air (http://www.londonair.org.uk/), go through the same process to fix up the data.

* If you don't want to work with a new air pollution dataset, use the SQL connection to download only the *ratified* data. Explore how the NO, NO2 and NOX readings vary together over time, using plots and any other methods you feel like trying out (covariance and correlation, perhaps?).

* Go and find another dataset to play with at the UK Open Datastore (http://data.gov.uk/data/search). There are some truly terrible, messy examples on there to get your teeth stuck into. 

